In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


## Read the data

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Read the data
df_full = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv', index_col='Id')
df_test_full =  pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')

In [3]:
df_full.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

## get the features with high correlation

In [4]:
#find the correlation between SalePrice and all the other coulmns
corr_df=df_full[df_full.columns[:]].corr()['SalePrice']
#soet by correlation value between SalePrice and all the other coulmns
corr_df=corr_df.sort_values(ascending=False)
# Convert to dataframe an get top 10 which have corr value more then 0.5
correlations = pd.DataFrame(corr_df).reset_index() 
high_correlations=correlations.values[0:11]
high_correlations

array([['SalePrice', 1.0],
       ['OverallQual', 0.7909816005838053],
       ['GrLivArea', 0.7086244776126515],
       ['GarageCars', 0.6404091972583519],
       ['GarageArea', 0.6234314389183622],
       ['TotalBsmtSF', 0.6135805515591943],
       ['1stFlrSF', 0.6058521846919153],
       ['FullBath', 0.5606637627484453],
       ['TotRmsAbvGrd', 0.5337231555820284],
       ['YearBuilt', 0.5228973328794967],
       ['YearRemodAdd', 0.5071009671113866]], dtype=object)

In [5]:
coul_with_high_corr=high_correlations[:,0]

In [6]:
df=df_full[coul_with_high_corr]
df.describe()

,SalePrice,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,180921.195890,6.099315,1515.463699,1.767123,472.980137,1057.429452,1162.626712,1.565068,6.517808,1971.267808,1984.865753
std,79442.502883,1.382997,525.480383,0.747315,213.804841,438.705324,386.587738,0.550916,1.625393,30.202904,20.645407
min,34900.000000,1.000000,334.000000,0.000000,0.000000,0.000000,334.000000,0.000000,2.000000,1872.000000,1950.000000
25%,129975.000000,5.000000,1129.500000,1.000000,334.500000,795.750000,882.000000,1.000000,5.000000,1954.000000,1967.000000
50%,163000.000000,6.000000,1464.000000,2.000000,480.000000,991.500000,1087.000000,2.000000,6.000000,1973.000000,1994.000000
75%,214000.000000,7.000000,1776.750000,2.000000,576.000000,1298.250000,1391.250000,2.000000,7.000000,2000.000000,2004.000000
max,755000.000000,10.000000,5642.000000,4.000000,1418.000000,6110.000000,4692.000000,3.000000,14.000000,2010.000000,2010.000000


### I will drop GarageArea as it's the same feture like GarageCars and GarageCars have higher correlation 


In [7]:
df.drop(['GarageArea'], axis=1, inplace=True)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
df

,SalePrice,OverallQual,GrLivArea,GarageCars,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd
Id,,,,,,,,,,
1,208500,7,1710,2,856,856,2,8,2003,2003
2,181500,6,1262,2,1262,1262,2,6,1976,1976
3,223500,7,1786,2,920,920,2,6,2001,2002
4,140000,7,1717,3,756,961,1,7,1915,1970
5,250000,8,2198,3,1145,1145,2,9,2000,2000
...,...,...,...,...,...,...,...,...,...,...
1456,175000,6,1647,2,953,953,2,7,1999,2000
1457,210000,6,2073,2,1542,2073,2,7,1978,1988
1458,266500,7,2340,1,1152,1188,2,9,1941,2006


In [9]:
y = df.SalePrice
df.drop(['SalePrice'], axis=1, inplace=True)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
X_train_final = df.copy()
X_test_final = df_test_full[coul_with_high_corr[1:]].copy()
X_test_final.drop(['GarageArea'], axis=1, inplace=True)
X_test_final

,OverallQual,GrLivArea,GarageCars,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd
Id,,,,,,,,,
1461,5,896,1.0,882.0,896,1,5,1961,1961
1462,6,1329,1.0,1329.0,1329,1,6,1958,1958
1463,5,1629,2.0,928.0,928,2,6,1997,1998
1464,6,1604,2.0,926.0,926,2,7,1998,1998
1465,8,1280,2.0,1280.0,1280,2,5,1992,1992
...,...,...,...,...,...,...,...,...,...
2915,4,1092,0.0,546.0,546,1,5,1970,1970
2916,4,1092,1.0,546.0,546,1,6,1970,1970
2917,5,1224,2.0,1224.0,1224,1,7,1960,1996


In [11]:
X_train_final

,OverallQual,GrLivArea,GarageCars,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd
Id,,,,,,,,,
1,7,1710,2,856,856,2,8,2003,2003
2,6,1262,2,1262,1262,2,6,1976,1976
3,7,1786,2,920,920,2,6,2001,2002
4,7,1717,3,756,961,1,7,1915,1970
5,8,2198,3,1145,1145,2,9,2000,2000
...,...,...,...,...,...,...,...,...,...
1456,6,1647,2,953,953,2,7,1999,2000
1457,6,2073,2,1542,2073,2,7,1978,1988
1458,7,2340,1,1152,1188,2,9,1941,2006


## Break off validation set from training data


In [12]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X_train_final, y,train_size=0.8, test_size=0.2,random_state=0)

## XGBRegressor model

In [13]:
from xgboost import XGBRegressor

xmodel=XGBRegressor(learning_rate=0.1,n_estimators=100)
xmodel.fit(X_train,y_train)

preds = xmodel.predict(X_valid)
print('RMSE:', mean_squared_error(np.log(y_valid), np.log(preds),squared=False))


RMSE: 0.14878004344935536


## RandomForestRegressor model

In [14]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(n_estimators=200, max_depth=10)
forest_model.fit(X_train, y_train)
predicted_random_forest = forest_model.predict(X_valid)
print('RMSE:', mean_squared_error(np.log(y_valid), np.log(predicted_random_forest),squared=False))

RMSE: 0.14836558391545335


### get the RMSE to different n_estimators 

In [15]:
RMSE_XGBRegressor=[]
RMSE_RandomForestRegressor=[]
for i in range(10,200,20):
    xmodel=XGBRegressor(n_estimators=i)
    xmodel.fit(X_train,y_train)
    preds = xmodel.predict(X_valid)
    RMSE_XGBRegressor.append(mean_squared_error(np.log(y_valid), np.log(preds),squared=False))

    forest_model = RandomForestRegressor(n_estimators=i)
    forest_model.fit(X_train, y_train)
    predicted_random_forest = forest_model.predict(X_valid)
    RMSE_RandomForestRegressor.append(mean_squared_error(np.log(y_valid), np.log(predicted_random_forest),squared=False))


## RMSE VS n_estimators 

In [16]:
import plotly.graph_objects as go

x =np.arange(10,200,20)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=RMSE_XGBRegressor,
    name = 'XGBRegressor', # Style name/legend entry with html tags
    connectgaps=True # override default to connect the gaps
))
fig.add_trace(go.Scatter(
    x=x,
    y=RMSE_RandomForestRegressor,
    name='RandomForestRegressor',
))


fig.update_layout(title='RMSE VS n_estimators ',
                   xaxis_title='n_estimators',
                   yaxis_title='RMSE')

fig.show()

> ### From the grarh I found that RandomForestRegressor with n_estimators = 170 have the least RMSE

In [17]:
forest_model = RandomForestRegressor(n_estimators=170, max_depth=10)
forest_model.fit(X_train, y_train)
predicted_random_forest = forest_model.predict(X_valid)
print('RMSE:', mean_squared_error(np.log(y_valid), np.log(predicted_random_forest),squared=False))

RMSE: 0.14996710314635256


In [18]:
X_test_final.fillna(0, inplace=True)

In [19]:
preds_test = forest_model.predict(X_test_final)

In [20]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test_final.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)